In [15]:
#IMPORTS
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import smtplib
import pandas as pd
import pyodbc

In [16]:
#CONNECT TO DATABASE
sql_conn = pyodbc.connect('Driver={SQL Server}; Server=localhost,1432; INSTANCE=MSSQLSERVER; Database=School; Trusted_Connection=yes; Encrypt=no')

In [17]:
query = '''SELECT p.FirstName, p.LastName, c.Title ,sg.Grade
FROM Person p
JOIN StudentGrade sg
ON p.PersonID = sg.StudentID
JOIN Course c
ON c.CourseID = sg.CourseID
WHERE p.Discriminator = 'Student'
ORDER BY p.LastName, p.FirstName;'''
df = pd.read_sql_query(query, sql_conn)

C:\Users\The user\AppData\Local\Temp\ipykernel_14012\3746325339.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, sql_conn)


In [19]:
#SEND A DATAFRAME IN AN EMAIL
msg = MIMEMultipart()
fromaddr = 'sfinkel@integralytic.com'
toaddr = "sfinkel@integralytic.com"
msg['From'] = fromaddr
msg["To"] = toaddr
msg['Subject'] = "Python email service lesson - example 1"

#Let's say course is an important table,, want to see its contents in email.
body = 'All students grades.'   #Any string/variable, no limit to how many
htmlEmail = """      <html>    
<head></head>
<body>
    {0}
</body>
</html>
""".format(df.to_html())
msg.attach(MIMEText(body, "plain"))
msg.attach(MIMEText(htmlEmail, 'html'))
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login(fromaddr, "kikmlhxgsockpzkl")
text = msg.as_string()
server.sendmail(fromaddr, toaddr, text)
server.quit()

(221,
 b'2.0.0 closing connection j8-20020a170906830800b009927d4d7a6bsm910033ejx.53 - gsmtp')

In [35]:
query1 = '''SELECT FirstName, LastName, HireDate
FROM Person
WHERE HireDate BETWEEN DATEADD(WEEK, -1, GETDATE()) AND GETDATE()
AND Discriminator = 'Instructor';'''
df1 = pd.read_sql_query(query1, sql_conn)

query2 = '''SELECT FirstName, LastName, EnrollmentDate
FROM Person
WHERE EnrollmentDate BETWEEN DATEADD(WEEK, -1, GETDATE()) AND GETDATE()
AND Discriminator = 'Student';'''
df2 = pd.read_sql_query(query2, sql_conn)


C:\Users\The user\AppData\Local\Temp\ipykernel_14012\4263828492.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql_query(query1, sql_conn)
C:\Users\The user\AppData\Local\Temp\ipykernel_14012\4263828492.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql_query(query2, sql_conn)


In [36]:
#SEND A DATAFRAME IN AN EMAIL
msg = MIMEMultipart()
fromaddr = 'sfinkel@integralytic.com'
toaddr = "sfinkel@integralytic.com"
msg['From'] = fromaddr
msg["To"] = toaddr
msg['Subject'] = "Weekly New Instructor/Student Report"

#Let's say course is an important table,, want to see its contents in email.
if df1.empty:
    body = 'No instructors were hired this week.'
    msg.attach(MIMEText(body, "plain"))
elif df1.shape[0] == 1:
    body = 'A new instructor was hired this week, Here are his details.'   #Any string/variable, no limit to how many
    htmlEmail = """      <html>    
    <head></head>
    <body>
        {0}
    </body>
    </html>
    """.format(df1.to_html())
    msg.attach(MIMEText(body, "plain"))
    msg.attach(MIMEText(htmlEmail, 'html'))
else:
    body = 'New instructors were hired this week, Here are their details.'   #Any string/variable, no limit to how many
    htmlEmail = """      <html>    
    <head></head>
    <body>
        {0}
    </body>
    </html>
    """.format(df1.to_html())
    msg.attach(MIMEText(body, "plain"))
    msg.attach(MIMEText(htmlEmail, 'html'))

htmlEmail2 = """      <html>    
    <head></head>
    <body>
        <br>
        <br>
    </body>
    </html>
    """
msg.attach(MIMEText(htmlEmail2, 'html'))
    

if df2.empty:
    body1 = 'No students were enrolled this week.'
    msg.attach(MIMEText(body1, "plain"))
elif df2.shape[0] == 1:
    body1 = 'A new student was enrolled this week, Here are his details.'   #Any string/variable, no limit to how many
    htmlEmail1 = """      <html>    
    <head></head>
    <body>
        {0}
    </body>
    </html>
    """.format(df2.to_html())
    msg.attach(MIMEText(body1, "plain"))
    msg.attach(MIMEText(htmlEmail1, 'html'))
else:
    body1 = 'New instructors were hired this week, Here are their details.'   #Any string/variable, no limit to how many
    htmlEmail1 = """      <html>    
    <head></head>
    <body>
        {0}
    </body>
    </html>
    """.format(df2.to_html())
    msg.attach(MIMEText(body1, "plain"))
    msg.attach(MIMEText(htmlEmail1, 'html'))
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login(fromaddr, "kikmlhxgsockpzkl")
text = msg.as_string()
server.sendmail(fromaddr, toaddr, text)
server.quit()

(221,
 b'2.0.0 closing connection n22-20020a17090625d600b00982a92a849asm1027411ejb.91 - gsmtp')